In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import keras as k
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, MaxPool2D, Conv2D, Flatten
from keras.models import Model
from keras import initializers

C:\Users\katie\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import cv2
import glob
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pydicom
import pandas as pd
import pickle
import random
import sys

import data_manipulation
import pickle

In [3]:
with open("data/dataset_train.obj", "rb") as f:
    train = pickle.load(f)
with open("data/dataset_val.obj", "rb") as f:
    val = pickle.load(f)

In [4]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape = (1024, 1024, 3))

In [5]:
class_maxpool1 = MaxPool2D(pool_size=(2,2), strides = None, padding='same', name = 'classifier_maxpool1')(vgg16.layers[-1].output)
class_conv2d_1 = Conv2D(filters = 256, kernel_size=(1,1), padding='same', name='classifier_conv2d_1', 
                        activation='relu')(class_maxpool1)
class_conv2d_2 = Conv2D(filters = 56, kernel_size=(1,1), padding='same', name='classifier_conv2d_2', 
                        activation='relu')(class_conv2d_1)
class_conv2d_3 = Conv2D(filters = 28, kernel_size=(1,1), padding='same', name='classifier_conv2d_3', 
                        activation='relu')(class_conv2d_2)
flatten_1 = Flatten()(class_conv2d_3)
class_dense_1 = Dense(units = 2, activation ='softmax')(flatten_1)


my_model = Model(input=vgg16.input, output = class_dense_1)


C:\Users\katie\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':


my_model.summary()

In [6]:
my_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [7]:
train_labels = train.labels_list()
val_labels = val.labels_list()

In [12]:
import data_generator
params = {'dim': (1024,1024),
          'batch_size': 64,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}
training_generator = data_generator.DataGenerator(train, range(train.size()), train_labels, **params)
validation_generator = data_generator.DataGenerator(val, range(val.size()), val_labels, **params)

In [13]:
my_model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False)
# #my_model.train(train, val,
#             learning_rate=data_manipulation.LEARNING_RATE,
#             epochs=1,
#             layers='all',
#             augmentation=None)  ## no need to augment yet
# my_model.fit(x=train, output=,epochs=,batch_size=)

Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Adam/mul_126 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Adam/beta_1/read, training/Adam/Variable_25/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss/mul/_421 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1956_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
